# 爬取大盤

#### 爬蟲的網址
1.臺灣證券交易所 (twe) <br>
2.證券櫃檯買賣中心 (otc) <br> 
3.公開資訊觀測站 (twse) <br>

In [15]:
import datetime
import requests
import pandas as pd
from io import StringIO

date = datetime.date(2018,1,3)
date_str = date.strftime('%Y%m%d')
res = requests.get("http://www.twse.com.tw/exchangeReport/MI_5MINS_INDEX?response=csv&date=" + 
                   date_str + "&_=1544020420045")

df = pd.read_csv(StringIO(res.text.replace("=","")), header=1, index_col='時間')

df = df.dropna(how='all', axis=0).dropna(how='all', axis=1)
df.index = pd.to_datetime(date.strftime('%Y %m %d ') + pd.Series(df.index))
df = df.apply(lambda s: s.astype(str).str.replace(",", "").astype(float))

df.head()

,發行量加權股價指數,未含金融保險股指數,未含電子股指數,未含金融電子股指數,水泥類指數,食品類指數,塑膠類指數,紡織纖維類指數,電機機械類指數,電器電纜類指數,...,電子通路類指數,資訊服務類指數,其他電子類指數,建材營造類指數,航運類指數,觀光類指數,金融保險類指數,貿易百貨類指數,油電燃氣類指數,其他類指數
時間,,,,,,,,,,,,,,,,,,,,,
2018-01-03 09:00:00,10710.73,9094.86,14537.98,12469.76,118.30,1650.04,284.81,491.68,195.92,52.67,...,120.67,88.49,101.61,261.65,69.72,134.95,1196.45,238.23,145.33,253.80
2018-01-03 09:00:05,10759.61,9139.70,14553.75,12478.29,118.30,1650.11,285.31,491.65,195.94,53.14,...,120.67,88.51,101.61,261.65,69.97,134.95,1198.94,238.23,145.33,253.80
2018-01-03 09:00:10,10765.54,9144.68,14564.15,12487.82,118.35,1650.11,285.54,492.46,196.11,53.18,...,120.67,88.51,101.86,261.81,70.09,134.95,1199.65,238.23,145.33,253.88
2018-01-03 09:00:15,10775.89,9155.36,14575.94,12504.18,118.35,1648.79,285.58,492.26,196.49,53.18,...,120.79,88.52,101.92,261.95,70.10,134.95,1199.13,239.05,146.53,254.06
2018-01-03 09:00:20,10777.92,9157.53,14577.97,12507.33,118.35,1649.48,285.51,492.88,196.75,53.15,...,120.86,88.51,101.94,262.05,70.13,134.95,1198.96,239.09,146.53,254.09


# GUI介面化

In [19]:
import sys, os
from finlab.crawler import commit
sys.path.append(os.pardir)

from finlab.crawler import (
    
    # 爬蟲們
    crawl_price,
    crawl_bargin,
    crawl_pe,
    crawl_monthly_report,
    crawl_finance_statement_by_date,
    crawl_twse_divide_ratio,
    crawl_otc_divide_ratio,
    crawl_twse_cap_reduction,
    crawl_otc_cap_reduction,
    crawl_benchmark,
    
    # 時間的range
    date_range,
    month_range,
    season_range,
    
    # GUI介面
    widget, out,
    
)

widget('price', crawl_price, date_range)
widget('bargin_report', crawl_bargin, date_range)
widget('pe', crawl_pe, date_range)
widget('monthly_report', crawl_monthly_report, month_range)
widget('financial_statement', crawl_finance_statement_by_date, season_range) # 財務報表
widget('twse_divide_ratio', crawl_twse_divide_ratio) # 除權除息計算結果表
widget('otc_divide_ratio', crawl_otc_divide_ratio) # 除權除息計算結果表
widget('twse_cap_reduction', crawl_twse_cap_reduction) # 股票減資恢復買賣參考價格
widget('otc_cap_reduction', crawl_otc_cap_reduction) # 股票減資恢復買賣參考價格

out.clear_output()
display(out)
commit()

Output(layout=Layout(border='1px solid black'))

已經成功commit過 twse_divide_ratio 了，跳過！
已經成功commit過 income_sheet 了，跳過！
已經成功commit過 twse_cap_reduction 了，跳過！
已經成功commit過 otc_cap_reduction 了，跳過！
已經成功commit過 bargin_report 了，跳過！
已經成功commit過 price 了，跳過！
已經成功commit過 pe 了，跳過！
已經成功commit過 benchmark 了，跳過！
已經成功commit過 cash_flows 了，跳過！
已經成功commit過 income_sheet_cumulate 了，跳過！
已經成功commit過 otc_divide_ratio 了，跳過！
已經成功commit過 monthly_report 了，跳過！
已經成功commit過 balance_sheet 了，跳過！


# AUTO update 每日自動執行更新

In [20]:
from finlab.crawler import table_date_range, update_table, to_pickle, commit
import datetime
import auto_update
from inspect import signature
from finlab.crawler import (

    # 爬蟲們
    crawl_price,
    crawl_bargin,
    crawl_pe,
    crawl_monthly_report,
    crawl_finance_statement_by_date,
    crawl_twse_divide_ratio,
    crawl_otc_divide_ratio,
    crawl_twse_cap_reduction,
    crawl_otc_cap_reduction,
    crawl_benchmark,
    
    # 時間的range
    date_range,
    month_range,
    season_range,
    
    # GUI介面
    widget, out,
)

def auto_update(table_name, crawl_function, time_range=None):

    sig = signature(crawl_function)
    print(table_name)

    if len(sig.parameters) != 0:
        first_date, last_date = table_date_range(table_name)
        dates = time_range(last_date, datetime.datetime.now())
        if dates:
            update_table(table_name, crawl_function, dates)
    else:
        df = crawl_function()
        to_pickle(df, table_name)
        

auto_update('price', crawl_price, date_range)
auto_update('bargin_report', crawl_bargin, date_range)
auto_update('pe', crawl_pe, date_range)
auto_update('monthly_report', crawl_monthly_report, month_range)
auto_update('financial_statement', crawl_finance_statement_by_date, season_range)
auto_update('twse_divide_ratio', crawl_twse_divide_ratio)
auto_update('otc_divide_ratio', crawl_otc_divide_ratio)
auto_update('twse_cap_reduction', crawl_twse_cap_reduction)
auto_update('otc_cap_reduction', crawl_otc_cap_reduction)

commit() #將資料格式轉換

twse_divide_ratio -- 資料日期
twse_divide_ratio -- 股票代號
twse_divide_ratio -- 股票名稱
twse_divide_ratio -- 除權息前收盤價
twse_divide_ratio -- 除權息參考價
twse_divide_ratio -- 權值+息值
twse_divide_ratio -- 權/息
twse_divide_ratio -- 漲停價格
twse_divide_ratio -- 跌停價格
twse_divide_ratio -- 開盤競價基準
twse_divide_ratio -- 減除股利參考價
twse_divide_ratio -- 詳細資料
twse_divide_ratio -- 最近一次申報資料 季別/日期
twse_divide_ratio -- 最近一次申報每股 (單位)淨值
twse_divide_ratio -- 最近一次申報每股 (單位)盈餘
twse_divide_ratio -- twse_divide_ratio
otc_divide_ratio -- 除權息日期
otc_divide_ratio -- 代號
otc_divide_ratio -- 名稱
otc_divide_ratio -- 除權息前收盤價
otc_divide_ratio -- 除權息參考價
otc_divide_ratio -- 權值
otc_divide_ratio -- 息值
otc_divide_ratio -- 權+息值
otc_divide_ratio -- 權/息
otc_divide_ratio -- 漲停價格
otc_divide_ratio -- 跌停價格
otc_divide_ratio -- 開盤競價基準
otc_divide_ratio -- 減除股利參考價
otc_divide_ratio -- 現金股利
otc_divide_ratio -- 每千股無償配股
otc_divide_ratio -- -
otc_divide_ratio -- 現金增資股數
otc_divide_ratio -- 現金增資認購價
otc_divide_ratio -- 公開承銷股數
otc_divide_ratio -- 員工認購股數
otc_divide_ratio -